In [1]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

sys.path.append('../')
from src.util.functions import Functions

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = '../data'
path_csv = '{path_data}/csv'.format(path_data=path_data)

csv_out_name = '{path_csv}/covid19-{level}.csv'.format(path_csv=path_csv, level=level)
pop_regions_name = '{path_csv}/popolazione-regioni.csv'.format(path_csv=path_csv)
icu_regions_name = '{path_csv}/terapie-intensive.csv'.format(path_csv=path_csv)
regions_name = '{path_csv}/regioni.csv'.format(path_csv=path_csv)

In [2]:
url_cum = Functions.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note,ingressi_terapia_intensiva,note_test,note_casi
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,0,NaN,NaN,0,5,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,0,NaN,NaN,0,1,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,0,NaN,NaN,0,10,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,0,NaN,NaN,18,148,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6547,2020-12-31T17:00:00,ITA,19,Sicilia,38.115697,13.362357,1069,171,1240,32628,...,2412,NaN,NaN,93644,1219132,812545.0,NaN,13.0,NaN,NaN
6548,2020-12-31T17:00:00,ITA,9,Toscana,43.769231,11.255889,838,150,988,8690,...,3673,NaN,NaN,120328,1883593,1061504.0,NaN,3.0,Positivi diagnosticati solo con test antigenic...,NaN
6549,2020-12-31T17:00:00,ITA,10,Umbria,43.106758,12.388247,259,44,303,3474,...,624,NaN,NaN,28960,503641,252395.0,NaN,3.0,NaN,NaN
6550,2020-12-31T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,75,2,77,334,...,379,NaN,NaN,7273,62687,37432.0,NaN,0.0,NaN,NaN


In [4]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'note']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,0,0,NaN,NaN,0,5,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,0,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,0,0,NaN,NaN,0,1,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,0,0,NaN,NaN,0,10,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,0,0,NaN,NaN,18,148,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6547,2020-12-31T17:00:00,19,Sicilia,1069,171,1240,32628,33868,481,1299,57364,2412,NaN,NaN,93644,1219132,812545.0,13.0,NaN,NaN
6548,2020-12-31T17:00:00,9,Toscana,838,150,988,8690,9678,-59,632,106977,3673,NaN,NaN,120328,1883593,1061504.0,3.0,Positivi diagnosticati solo con test antigenic...,NaN
6549,2020-12-31T17:00:00,10,Umbria,259,44,303,3474,3777,71,300,24559,624,NaN,NaN,28960,503641,252395.0,3.0,NaN,NaN
6550,2020-12-31T17:00:00,2,Valle d'Aosta,75,2,77,334,411,5,29,6483,379,NaN,NaN,7273,62687,37432.0,0.0,NaN,NaN


In [5]:
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = Functions.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2020-12-31', '2020-12-30')

In [6]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2020-12-31T17:00:00,1,Piemonte,2895,190,3085,25172,28257,-702,1367,161649,7922,NaN,NaN,197828,1682529,984809.0,7.0,NaN,NaN
1,2020-12-31T17:00:00,2,Valle d'Aosta,75,2,77,334,411,5,29,6483,379,NaN,NaN,7273,62687,37432.0,0.0,NaN,NaN
2,2020-12-31T17:00:00,3,Lombardia,3437,489,3926,50697,54623,1627,3859,399157,25123,NaN,NaN,478903,4859567,2608285.0,46.0,NaN,NaN
3,2020-12-31T17:00:00,5,Veneto,2698,361,3059,88014,91073,1050,4800,156263,6539,NaN,NaN,253875,3300987,1214443.0,31.0,NaN,NaN
4,2020-12-31T17:00:00,6,Friuli Venezia Giulia,645,62,707,11091,11798,151,877,36587,1642,NaN,NaN,50027,933082,341139.0,14.0,NaN,NaN
5,2020-12-31T17:00:00,7,Liguria,694,65,759,4861,5620,-8,472,51958,2891,NaN,NaN,60469,711036,323257.0,5.0,I 160617 test antigenici rapidi sono riferiti ...,NaN
6,2020-12-31T17:00:00,8,Emilia-Romagna,2629,233,2862,54484,57346,1289,2116,106428,7738,NaN,NaN,171512,2572140,1227556.0,20.0,NaN,NaN
7,2020-12-31T17:00:00,9,Toscana,838,150,988,8690,9678,-59,632,106977,3673,NaN,NaN,120328,1883593,1061504.0,3.0,Positivi diagnosticati solo con test antigenic...,NaN
8,2020-12-31T17:00:00,10,Umbria,259,44,303,3474,3777,71,300,24559,624,NaN,NaN,28960,503641,252395.0,3.0,NaN,NaN
9,2020-12-31T17:00:00,11,Marche,470,62,532,10646,11178,606,703,28875,1571,NaN,NaN,41624,532497,315319.0,4.0,NaN,NaN


In [7]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2020-12-30T17:00:00,1,Piemonte,2957,192,3149,25810,28959,-424,1046,159609,7893,NaN,NaN,196461,1673896,981588.0,7.0,NaN,NaN
1,2020-12-30T17:00:00,2,Valle d'Aosta,72,2,74,332,406,-7,14,6459,379,NaN,NaN,7244,62480,37323.0,2.0,NaN,NaN
2,2020-12-30T17:00:00,3,Lombardia,3617,481,4098,48898,52996,-1422,1673,397010,25038,NaN,NaN,475044,4826709,2597894.0,21.0,NaN,NaN
3,2020-12-30T17:00:00,5,Veneto,2711,358,3069,86954,90023,-342,2986,152642,6410,NaN,NaN,249075,3276810,1208044.0,36.0,NaN,NaN
4,2020-12-30T17:00:00,6,Friuli Venezia Giulia,641,59,700,10947,11647,-130,803,35876,1627,NaN,NaN,49150,924819,338891.0,5.0,NaN,A seguito della periodica verifica dei casi se...
5,2020-12-30T17:00:00,7,Liguria,712,63,775,4853,5628,-131,415,51489,2880,NaN,NaN,59997,706883,322079.0,2.0,I 155891 test antigenici rapidi sono riferiti ...,NaN
6,2020-12-30T17:00:00,8,Emilia-Romagna,2641,230,2871,53186,56057,-755,1427,105656,7683,NaN,NaN,169396,2556405,1221526.0,19.0,NaN,NaN
7,2020-12-30T17:00:00,9,Toscana,869,160,1029,8708,9737,-278,460,106304,3655,NaN,NaN,119696,1872240,1057968.0,10.0,Positivi diagnosticati solo con test antigenic...,NaN
8,2020-12-30T17:00:00,10,Umbria,256,44,300,3406,3706,138,347,24336,618,NaN,NaN,28660,500003,251329.0,8.0,NaN,NaN
9,2020-12-30T17:00:00,11,Marche,468,62,530,10042,10572,443,585,28792,1557,NaN,NaN,40921,526915,311670.0,7.0,NaN,NaN


In [8]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2020-12-31T17:00:00,1,Piemonte,2895,190,3085,25172,28257,-702,1367,161649,7922,NaN,NaN,197828,1682529,984809.0,7.0,NaN,NaN
1,2020-12-31T17:00:00,2,Valle d'Aosta,75,2,77,334,411,5,29,6483,379,NaN,NaN,7273,62687,37432.0,0.0,NaN,NaN
2,2020-12-31T17:00:00,3,Lombardia,3437,489,3926,50697,54623,1627,3859,399157,25123,NaN,NaN,478903,4859567,2608285.0,46.0,NaN,NaN
3,2020-12-31T17:00:00,4,Trentino-Alto Adige,539,68,607,11924,12531,216,580,37122,1681,0.0,0.0,51334,812052,314154.0,7.0,0,NaN
4,2020-12-31T17:00:00,5,Veneto,2698,361,3059,88014,91073,1050,4800,156263,6539,NaN,NaN,253875,3300987,1214443.0,31.0,NaN,NaN
5,2020-12-31T17:00:00,6,Friuli Venezia Giulia,645,62,707,11091,11798,151,877,36587,1642,NaN,NaN,50027,933082,341139.0,14.0,NaN,NaN
6,2020-12-31T17:00:00,7,Liguria,694,65,759,4861,5620,-8,472,51958,2891,NaN,NaN,60469,711036,323257.0,5.0,I 160617 test antigenici rapidi sono riferiti ...,NaN
7,2020-12-31T17:00:00,8,Emilia-Romagna,2629,233,2862,54484,57346,1289,2116,106428,7738,NaN,NaN,171512,2572140,1227556.0,20.0,NaN,NaN
8,2020-12-31T17:00:00,9,Toscana,838,150,988,8690,9678,-59,632,106977,3673,NaN,NaN,120328,1883593,1061504.0,3.0,Positivi diagnosticati solo con test antigenic...,NaN
9,2020-12-31T17:00:00,10,Umbria,259,44,303,3474,3777,71,300,24559,624,NaN,NaN,28960,503641,252395.0,3.0,NaN,NaN


In [9]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
df_r_0.sort_values(by='codice_regione').reset_index()

,index,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,...,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,0,2020-12-30T17:00:00,1,Piemonte,2957,192,3149,25810,28959,-424,...,159609,7893,NaN,NaN,196461,1673896,981588.0,7.0,NaN,NaN
1,1,2020-12-30T17:00:00,2,Valle d'Aosta,72,2,74,332,406,-7,...,6459,379,NaN,NaN,7244,62480,37323.0,2.0,NaN,NaN
2,2,2020-12-30T17:00:00,3,Lombardia,3617,481,4098,48898,52996,-1422,...,397010,25038,NaN,NaN,475044,4826709,2597894.0,21.0,NaN,NaN
3,19,2020-12-30T17:00:00,4,Trentino-Alto Adige,548,67,615,11700,12315,-123,...,36775,1664,0.0,0.0,50754,807454,313197.0,6.0,0,NaN
4,3,2020-12-30T17:00:00,5,Veneto,2711,358,3069,86954,90023,-342,...,152642,6410,NaN,NaN,249075,3276810,1208044.0,36.0,NaN,NaN
5,4,2020-12-30T17:00:00,6,Friuli Venezia Giulia,641,59,700,10947,11647,-130,...,35876,1627,NaN,NaN,49150,924819,338891.0,5.0,NaN,A seguito della periodica verifica dei casi se...
6,5,2020-12-30T17:00:00,7,Liguria,712,63,775,4853,5628,-131,...,51489,2880,NaN,NaN,59997,706883,322079.0,2.0,I 155891 test antigenici rapidi sono riferiti ...,NaN
7,6,2020-12-30T17:00:00,8,Emilia-Romagna,2641,230,2871,53186,56057,-755,...,105656,7683,NaN,NaN,169396,2556405,1221526.0,19.0,NaN,NaN
8,7,2020-12-30T17:00:00,9,Toscana,869,160,1029,8708,9737,-278,...,106304,3655,NaN,NaN,119696,1872240,1057968.0,10.0,Positivi diagnosticati solo con test antigenic...,NaN
9,8,2020-12-30T17:00:00,10,Umbria,256,44,300,3406,3706,138,...,24336,618,NaN,NaN,28660,500003,251329.0,8.0,NaN,NaN


In [10]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi,popolazione
0,2020-12-31T17:00:00,1,Piemonte,2895,190,3085,25172,28257,-702,1367,...,7922,NaN,NaN,197828,1682529,984809.0,7.0,NaN,NaN,4341375
1,2020-12-31T17:00:00,2,Valle d'Aosta,75,2,77,334,411,5,29,...,379,NaN,NaN,7273,62687,37432.0,0.0,NaN,NaN,125501
2,2020-12-31T17:00:00,3,Lombardia,3437,489,3926,50697,54623,1627,3859,...,25123,NaN,NaN,478903,4859567,2608285.0,46.0,NaN,NaN,10103969
3,2020-12-31T17:00:00,4,Trentino-Alto Adige,539,68,607,11924,12531,216,580,...,1681,0.0,0.0,51334,812052,314154.0,7.0,0,NaN,1074819
4,2020-12-31T17:00:00,5,Veneto,2698,361,3059,88014,91073,1050,4800,...,6539,NaN,NaN,253875,3300987,1214443.0,31.0,NaN,NaN,4907704
5,2020-12-31T17:00:00,6,Friuli Venezia Giulia,645,62,707,11091,11798,151,877,...,1642,NaN,NaN,50027,933082,341139.0,14.0,NaN,NaN,1211357
6,2020-12-31T17:00:00,7,Liguria,694,65,759,4861,5620,-8,472,...,2891,NaN,NaN,60469,711036,323257.0,5.0,I 160617 test antigenici rapidi sono riferiti ...,NaN,1543127
7,2020-12-31T17:00:00,8,Emilia-Romagna,2629,233,2862,54484,57346,1289,2116,...,7738,NaN,NaN,171512,2572140,1227556.0,20.0,NaN,NaN,4467118
8,2020-12-31T17:00:00,9,Toscana,838,150,988,8690,9678,-59,632,...,3673,NaN,NaN,120328,1883593,1061504.0,3.0,Positivi diagnosticati solo con test antigenic...,NaN,3722729
9,2020-12-31T17:00:00,10,Umbria,259,44,303,3474,3777,71,300,...,624,NaN,NaN,28960,503641,252395.0,3.0,NaN,NaN,880285


In [11]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, on='denominazione_regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi,popolazione,totale_posti_terapia_intensiva
0,2020-12-31T17:00:00,1,Piemonte,2895,190,3085,25172,28257,-702,1367,...,NaN,NaN,197828,1682529,984809.0,7.0,NaN,NaN,4341375,575
1,2020-12-31T17:00:00,2,Valle d'Aosta,75,2,77,334,411,5,29,...,NaN,NaN,7273,62687,37432.0,0.0,NaN,NaN,125501,20
2,2020-12-31T17:00:00,3,Lombardia,3437,489,3926,50697,54623,1627,3859,...,NaN,NaN,478903,4859567,2608285.0,46.0,NaN,NaN,10103969,1036
3,2020-12-31T17:00:00,4,Trentino-Alto Adige,539,68,607,11924,12531,216,580,...,0.0,0.0,51334,812052,314154.0,7.0,0,NaN,1074819,106
4,2020-12-31T17:00:00,5,Veneto,2698,361,3059,88014,91073,1050,4800,...,NaN,NaN,253875,3300987,1214443.0,31.0,NaN,NaN,4907704,1016
5,2020-12-31T17:00:00,6,Friuli Venezia Giulia,645,62,707,11091,11798,151,877,...,NaN,NaN,50027,933082,341139.0,14.0,NaN,NaN,1211357,180
6,2020-12-31T17:00:00,7,Liguria,694,65,759,4861,5620,-8,472,...,NaN,NaN,60469,711036,323257.0,5.0,I 160617 test antigenici rapidi sono riferiti ...,NaN,1543127,209
7,2020-12-31T17:00:00,8,Emilia-Romagna,2629,233,2862,54484,57346,1289,2116,...,NaN,NaN,171512,2572140,1227556.0,20.0,NaN,NaN,4467118,634
8,2020-12-31T17:00:00,9,Toscana,838,150,988,8690,9678,-59,632,...,NaN,NaN,120328,1883593,1061504.0,3.0,Positivi diagnosticati solo con test antigenic...,NaN,3722729,523
9,2020-12-31T17:00:00,10,Umbria,259,44,303,3474,3777,71,300,...,NaN,NaN,28960,503641,252395.0,3.0,NaN,NaN,880285,117


In [12]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [13]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['totale_posti_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['totale_posti_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2020-12-31T17:00:00,5,Veneto,2698,361,3059,88014,91073,1050,4800,...,35.53,1060,1814,24177,6399.0,129,3621,19.85,0.09781,5.17
1,2020-12-31T17:00:00,3,Lombardia,3437,489,3926,50697,54623,1627,3859,...,47.20,1799,2186,32858,10391.0,85,2147,11.74,0.03819,4.74
2,2020-12-31T17:00:00,8,Emilia-Romagna,2629,233,2862,54484,57346,1289,2116,...,36.75,1298,689,15735,6030.0,55,772,13.45,0.04737,3.84
3,2020-12-31T17:00:00,12,Lazio,2789,303,3092,72081,75173,710,1767,...,35.77,729,434,16301,6272.0,73,984,10.84,0.03013,2.78
4,2020-12-31T17:00:00,16,Puglia,1490,129,1619,51383,53002,262,1661,...,34.96,268,191,10836,4212.0,21,1378,15.33,0.04144,2.27
5,2020-12-31T17:00:00,15,Campania,1325,113,1438,75817,77255,-356,1554,...,22.38,-335,624,19003,14271.0,32,1878,8.18,0.02686,3.28
6,2020-12-31T17:00:00,1,Piemonte,2895,190,3085,25172,28257,-702,1367,...,33.04,-638,321,8633,3221.0,29,2040,15.83,0.03149,4.56
7,2020-12-31T17:00:00,19,Sicilia,1069,171,1240,32628,33868,481,1299,...,29.08,492,215,7308,4385.0,31,787,17.78,0.02615,1.88
8,2020-12-31T17:00:00,6,Friuli Venezia Giulia,645,62,707,11091,11798,151,877,...,34.44,144,74,8263,2248.0,15,711,10.61,0.07240,4.13
9,2020-12-31T17:00:00,11,Marche,470,62,532,10646,11178,606,703,...,43.36,604,118,5582,3649.0,14,83,12.59,0.04630,2.74


In [14]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [15]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'totale_posti_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,totale_posti_terapia_intensiva,saturazione_terapia_intensiva
0,Trentino-Alto Adige,68,38,106,64.15
1,Lombardia,489,547,1036,47.20
2,Marche,62,81,143,43.36
3,Umbria,44,73,117,37.61
4,Emilia-Romagna,233,401,634,36.75
5,Lazio,303,544,847,35.77
6,Veneto,361,655,1016,35.53
7,Puglia,129,240,369,34.96
8,Friuli Venezia Giulia,62,118,180,34.44
9,Piemonte,190,385,575,33.04


In [16]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['variazione_totale_positivi'] / (df['totale_positivi'] - df['variazione_totale_positivi'])).replace(np.inf, 0) * 100).round(2)
    return df.fillna(0)

In [17]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../data/csv/regioni/covid19-veneto.csv
Aggiornato ../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../data/csv/regioni/covid19-liguria.csv
Aggiornato ../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../data/csv/regioni/covid19-toscana.csv
Aggiornato ../data/csv/regioni/covid19-umbria.csv
Aggiornato ../data/csv/regioni/covid19-marche.csv
Aggiornato ../data/csv/regioni/covid19-lazio.csv
Aggiornato ../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../data/csv/regioni/covid19-molise.csv
Aggiornato ../data/csv/regioni/covid19-campania.csv
Aggiornato ../data/csv/regioni/covid19-puglia.csv
Aggiornato ../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../data/csv/regioni/covid19-calabria.csv
Aggiornato ../data/csv/regioni/covid19-sicilia.csv
Aggiornato ../data/csv/regioni/covid19-sardegna.csv


In [18]:
df_tb = df_rd[df_rd.codice_regione > 20].reset_index(drop=True)
df_tb

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2020-02-24T18:00:00,21,P.A. Bolzano,0,0,0,0,0,0,0,0,0,NaN,NaN,0,1,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,22,P.A. Trento,0,0,0,0,0,0,0,0,0,NaN,NaN,0,3,NaN,NaN,NaN,NaN
2,2020-02-25T18:00:00,21,P.A. Bolzano,1,0,1,0,1,1,1,0,0,NaN,NaN,1,1,NaN,NaN,NaN,NaN
3,2020-02-25T18:00:00,22,P.A. Trento,0,0,0,0,0,0,0,0,0,NaN,NaN,0,3,NaN,NaN,NaN,NaN
4,2020-02-26T18:00:00,21,P.A. Bolzano,1,0,1,0,1,0,0,0,0,NaN,NaN,1,1,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
619,2020-12-29T17:00:00,22,P.A. Trento,380,46,426,1334,1760,-61,175,18637,926,NaN,NaN,21323,445114,150638.0,4.0,NaN,"Dei 175 casi odierni, 83 sono molecolari con p..."
620,2020-12-30T17:00:00,21,P.A. Bolzano,168,22,190,10412,10602,-76,220,17901,731,NaN,NaN,29234,359695,162050.0,1.0,NaN,NaN
621,2020-12-30T17:00:00,22,P.A. Trento,380,45,425,1288,1713,-47,197,18874,933,NaN,NaN,21520,447759,151147.0,5.0,NaN,"Dei 197 casi odierni, 112 sono molecolari con ..."
622,2020-12-31T17:00:00,21,P.A. Bolzano,159,26,185,10484,10669,67,260,18086,739,NaN,NaN,29494,361459,162616.0,4.0,NaN,NaN


In [19]:
# Initialize df_taa
df_taa = Functions.replace_bt_with_taa(df_tb[:2])
for i in range(2, df_tb.shape[0], 2):
    df_taa = df_taa.append(Functions.replace_bt_with_taa(df_tb[i:i+2]), ignore_index=True)
df_taa = add_cols(df_taa)
df_taa

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,nuovi_deceduti,nuovi_tamponi,nuovi_casi_testati,variazione_isolamento_domiciliare,ti_su_to,to_su_tp,np_su_nt,np_su_nct,var_ti_perc,var_pos_perc
1,2020-02-25T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,1,1,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
2,2020-02-26T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
3,2020-02-27T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,30.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
4,2020-02-28T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
5,2020-02-29T17:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,25.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307,2020-12-27T17:00:00,4,Trentino-Alto Adige,510,65,575,11780,12355,63,143,...,7.0,1376.0,337.0,39.0,11.30,4.65,10.39,42.43,0.00,0.51
308,2020-12-28T17:00:00,4,Trentino-Alto Adige,527,65,592,11836,12428,73,219,...,14.0,1850.0,552.0,56.0,10.98,4.76,11.84,39.67,0.00,0.59
309,2020-12-29T17:00:00,4,Trentino-Alto Adige,516,68,584,11854,12438,10,266,...,19.0,2852.0,654.0,18.0,11.64,4.70,9.33,40.67,4.62,0.08
310,2020-12-30T17:00:00,4,Trentino-Alto Adige,548,67,615,11700,12315,-123,417,...,9.0,5003.0,1074.0,-154.0,10.89,4.99,8.33,38.83,-1.47,-0.99


In [20]:
region_name = df_taa.denominazione_regione.iloc[0].lower()
csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
df_taa.to_csv(csv_out_region_name, index=False)
print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-trentino-alto adige.csv
